In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [ ]:

# pip install mbutil

In [84]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

os.system('tippecanoe -Z0 -z5 -f -o tiles/countries.mbtiles --coalesce-densest-as-needed --generate-id '+ floc)


For layer 0, using name "county"
216 features, 1304369 bytes of geometry, 16 bytes of separate metadata, 10662 bytes of string pool
  97.1%  5/15/10  


0

In [85]:
base  = gpd.read_file('data/OA2011.geojson')[['OA11CD', 'LAD11CD',
       'geometry']]
centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

base.to_file("tmp/OA11.geojson", driver="GeoJSON")
centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

os.system('tippecanoe -Z6 -z12 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

For layer 0, using name "OA11"
For layer 1, using name "centroids11"
215786 features, 22300915 bytes of geometry, 1960 bytes of separate metadata, 2377501 bytes of string pool
tile 6/32/21 size is 554829 with detail 12, >500000    
Going to try keeping the sparsest 81.11% of the features to make it fit
tile 6/32/21 size is 532122 with detail 12, >500000    
Going to try keeping the sparsest 68.59% of the features to make it fit
tile 6/31/20 size is 1660791 with detail 12, >500000    
Going to try keeping the sparsest 27.10% of the features to make it fit
tile 6/32/21 size is 501192 with detail 12, >500000    
Going to try keeping the sparsest 61.58% of the features to make it fit
tile 6/31/21 size is 1736401 with detail 12, >500000    
Going to try keeping the sparsest 25.92% of the features to make it fit
tile 6/31/20 size is 951030 with detail 12, >500000    
Going to try keeping the sparsest 12.82% of the features to make it fit
tile 6/31/21 size is 1021684 with detail 12, >500000  

0

In [ ]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))




In [63]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles')

Tile 6/31/20 size is 517461, >500000. Skipping this tile
.Tile 6/31/21 size is 539867, >500000. Skipping this tile


0

In [41]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

SyntaxError: invalid syntax (1242424097.py, line 1)

In [83]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf %')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf')
# os.system("cd ./tiles/tileserver && find . -type f -exec mv '{}' '{}'.pbf \;")

DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:./tiles/merged.mbtiles --> ./tiles/tileserver
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:13 / 177 tiles exported
DEBUG:mbutil.util

gzip -d -r -S .pbf ./9/256/167.pbf
gzip -d -r -S .pbf ./9/248/169.pbf
gzip -d -r -S .pbf ./9/248/170.pbf
gzip -d -r -S .pbf ./9/248/172.pbf
gzip -d -r -S .pbf ./9/248/173.pbf
gzip -d -r -S .pbf ./9/246/173.pbf
gzip -d -r -S .pbf ./9/247/169.pbf
gzip -d -r -S .pbf ./9/247/173.pbf
gzip -d -r -S .pbf ./9/249/169.pbf
gzip -d -r -S .pbf ./9/249/168.pbf
gzip -d -r -S .pbf ./9/249/171.pbf
gzip -d -r -S .pbf ./9/249/165.pbf
gzip -d -r -S .pbf ./9/249/170.pbf
gzip -d -r -S .pbf ./9/249/166.pbf
gzip -d -r -S .pbf ./9/249/172.pbf
gzip -d -r -S .pbf ./9/249/173.pbf
gzip -d -r -S .pbf ./9/249/167.pbf
gzip -d -r -S .pbf ./9/253/169.pbf
gzip -d -r -S .pbf ./9/253/168.pbf
gzip -d -r -S .pbf ./9/253/163.pbf
gzip -d -r -S .pbf ./9/253/162.pbf
gzip -d -r -S .pbf ./9/253/160.pbf
gzip -d -r -S .pbf ./9/253/161.pbf
gzip -d -r -S .pbf ./9/253/171.pbf
gzip -d -r -S .pbf ./9/253/165.pbf
gzip -d -r -S .pbf ./9/253/159.pbf
gzip -d -r -S .pbf ./9/253/164.pbf
gzip -d -r -S .pbf ./9/253/170.pbf
gzip -d -r -S .pbf .

mv ./8/127/86 ./8/127/86.pbf
mv ./8/127/83 ./8/127/83.pbf
mv ./8/127/84 ./8/127/84.pbf
mv ./8/127/85 ./8/127/85.pbf
mv ./8/127/82 ./8/127/82.pbf
mv ./8/126/80 ./8/126/80.pbf
mv ./8/126/81 ./8/126/81.pbf
mv ./8/126/86 ./8/126/86.pbf
mv ./8/126/79 ./8/126/79.pbf
mv ./8/126/83 ./8/126/83.pbf
mv ./8/126/84 ./8/126/84.pbf
mv ./8/126/85 ./8/126/85.pbf
mv ./8/126/82 ./8/126/82.pbf
mv ./8/128/86 ./8/128/86.pbf
mv ./8/128/83 ./8/128/83.pbf
mv ./8/128/84 ./8/128/84.pbf
mv ./8/128/85 ./8/128/85.pbf
mv ./8/128/82 ./8/128/82.pbf
mv ./8/123/86 ./8/123/86.pbf
mv ./8/124/86 ./8/124/86.pbf
mv ./8/124/83 ./8/124/83.pbf
mv ./8/124/84 ./8/124/84.pbf
mv ./8/124/85 ./8/124/85.pbf
mv ./8/124/82 ./8/124/82.pbf
mv ./8/125/80 ./8/125/80.pbf
mv ./8/125/81 ./8/125/81.pbf
mv ./8/125/86 ./8/125/86.pbf
mv ./8/125/83 ./8/125/83.pbf
mv ./8/125/84 ./8/125/84.pbf
mv ./8/125/85 ./8/125/85.pbf
mv ./8/125/82 ./8/125/82.pbf
mv ./4/7/4 ./4/7/4.pbf
mv ./4/7/5 ./4/7/5.pbf
mv ./4/8/5 ./4/8/5.pbf
mv ./3/4/2 ./3/4/2.pbf
mv ./3/3/

0

In [78]:
os.system("cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % % %")

% ./9/257/169.pbf
xargs: %: No such file or directory


32512